In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import requests
import numpy as np
import pandas as pd
import time
import re

# Установление связи с сайтом

In [3]:
# Ссылка на страницу с книгами

page_link = 'https://exler.ru/films/?sort=-rating_exler&page=1'

In [4]:
# Загрузим фейкового юзерагента

from fake_useragent import UserAgent

UserAgent().safari

'Mozilla/5.0 (iPad; U; CPU OS 3_2 like Mac OS X; es-es) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B360 Safari/531.21.10'

In [5]:
# Запрос к сайту с использованием фейкового юзерагента

response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Наш запрос удался. Посмотрим что он выводит

In [6]:
html = response.content
html[:1000]

b'<!DOCTYPE html>\n<html lang="ru-RU">\n<head>\n        <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <meta name="format-detection" content="telephone=no">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <title>\xd0\x9a\xd0\xb8\xd0\xbd\xd0\xbe\xd1\x80\xd0\xb5\xd1\x86\xd0\xb5\xd0\xbd\xd0\xb7\xd0\xb8\xd0\xb8</title>\n        <link href="https://fonts.googleapis.com/css?family=PT+Sans:400,400i,700,700i&display=swap&subset=cyrillic,cyrillic-ext,latin-ext" rel="stylesheet">\n    <link href="/css/bootstrap.css" rel="stylesheet">\n<link href="/css/breakpoints.css" rel="stylesheet">\n<link href="/css/jquery.scrollbar.css" rel="stylesheet">\n<link href="/js/fancybox/jquery.fancybox.css" rel="stylesheet">\n<link href="/css/jquery.formstyler.css" rel="stylesheet">\n<link href="/css/slick.css" rel="stylesheet">\n<link href="/css/style.css?v5" rel="stylesheet">\n<link href="/css/

In [7]:
# Импортируем суп и через него посмотрим на вывод нашего запроса

from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify()[:2000])

<!DOCTYPE html>
<html lang="ru-RU">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Кинорецензии
  </title>
  <link href="https://fonts.googleapis.com/css?family=PT+Sans:400,400i,700,700i&amp;display=swap&amp;subset=cyrillic,cyrillic-ext,latin-ext" rel="stylesheet"/>
  <link href="/css/bootstrap.css" rel="stylesheet"/>
  <link href="/css/breakpoints.css" rel="stylesheet"/>
  <link href="/css/jquery.scrollbar.css" rel="stylesheet"/>
  <link href="/js/fancybox/jquery.fancybox.css" rel="stylesheet"/>
  <link href="/css/jquery.formstyler.css" rel="stylesheet"/>
  <link href="/css/slick.css" rel="stylesheet"/>
  <link href="/css/style.css?v5" rel="stylesheet"/>
  <link href="/css/custom.css?v5" rel="stylesheet"/>
  <link href="/twentytwenty/css/twentytwenty.css"

In [8]:
soup.html.head.title

<title>Кинорецензии</title>

# Получание ссылок на следующие страницы

In [20]:
# Для начала найдём ссылку на след. страницу

obj = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['film-item'])
obj

<div class="film-item">
<div class="film-item-block">
<div class="film-item-img crop-film-item-img">
<a href="/films/nastoyashij-detektiv-2.htm">
<img alt="" src="/films/images/07-12-2015/1.jpg"/> <div class="custom-item-mark">
                            9.0                        </div>
</a>
</div>
<div class="film-title-header">
<div class="film-title">
<a href="/films/nastoyashij-detektiv-2.htm">
                    Настоящий детектив 2                                          <span>/</span> True Detective 2                                    </a>
</div>
<div class="exler-reit">Рейтинг Экслера: <b>9.0</b></div>
</div>
<div class="film-info">
<dl class="dl-horizontal">
<dt>
                        Режиссер:
                    </dt>
<dd>
<a href="/films/cast/Nik_Piccolatto">Ник Пиццолатто</a> </dd>
</dl>
<dl class="dl-horizontal">
<dt>
                        В ролях:
                    </dt>
<dd>
<a href="/films/cast/Dzhejms_Frejn">Джеймс Фрейн</a>, <a href="/films/cast/Afemo_Omil

In [24]:
# Теперь попробуем вытащить ссылку 

link = obj.find('div', class_ = 'film-item-img crop-film-item-img').find('a')['href']
link

'/films/nastoyashij-detektiv-2.htm'

Код работает, первая ссылка получена. Теперь получим остальные ссылки на этой странице

In [28]:
# Спарсим все данные на странице

all_obj = soup.findAll(lambda tag: tag.name == 'div' and tag.get('class') == ['film-item'])
all_obj

[<div class="film-item">
 <div class="film-item-block">
 <div class="film-item-img crop-film-item-img">
 <a href="/films/nastoyashij-detektiv-2.htm">
 <img alt="" src="/films/images/07-12-2015/1.jpg"/> <div class="custom-item-mark">
                             9.0                        </div>
 </a>
 </div>
 <div class="film-title-header">
 <div class="film-title">
 <a href="/films/nastoyashij-detektiv-2.htm">
                     Настоящий детектив 2                                          <span>/</span> True Detective 2                                    </a>
 </div>
 <div class="exler-reit">Рейтинг Экслера: <b>9.0</b></div>
 </div>
 <div class="film-info">
 <dl class="dl-horizontal">
 <dt>
                         Режиссер:
                     </dt>
 <dd>
 <a href="/films/cast/Nik_Piccolatto">Ник Пиццолатто</a> </dd>
 </dl>
 <dl class="dl-horizontal">
 <dt>
                         В ролях:
                     </dt>
 <dd>
 <a href="/films/cast/Dzhejms_Frejn">Джеймс Фрейн</a>, <a

In [29]:
# Вытащим все ссылки на странице

links = [] # Пустой список для хранения ссылок

for obj in all_obj:
    img_div = obj.find('div', class_ = 'film-item-img crop-film-item-img')
    
    if img_div:
        link = img_div.find('a')['href']
        links.append(link)

print(links)

['/films/nastoyashij-detektiv-2.htm', '/films/nastoyashij-detektiv-1.htm', '/films/horas-i-pit.htm', '/films/hod-korolevy.htm', '/films/roma-1.htm', '/films/bezumnyj-maks-doroga-yarosti.htm', '/films/chernobyl.htm', '/films/delo-richarda-dzhuella.htm', '/films/vo-vse-tyazhkie.htm', '/films/udivitelnaya-missis-mejzel.htm']


In [30]:
# Общее кол-во ссылок на странице

len(links)

10

Видно что код работает и мы получаем нужные ссылки. Теперь обернём всё в функцию и пробежимся по всем страницам

In [96]:
def getPageLinks(page_number):
    '''
        Возвращает список ссылок на фильмы, получанные с текущей страницы
    '''
    
    # составляем ссылку на страницу поиска
    page_link = 'https://exler.ru/films/?sort=-rating_exler&page={}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers = {'User-Agent': UserAgent().safari})
    
    if not response.ok:
        # если сервер отказал вернём пустой лист для текущей страницы
        return []
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    
    # получаем данные со страниц
    all_obj = soup.findAll(lambda tag: tag.name == 'div' and tag.get('class') == ['film-item'])
    
    # получаем ссылки из данных
    links = [] 

    for obj in all_obj:
        img_div = obj.find('div', class_ = 'film-item-img crop-film-item-img')
    
        if img_div:
            link = img_div.find('a')['href']
            links.append(link)
            
    film_links = ['https://exler.ru' + link for link in links]
            
    return film_links

In [98]:
# Протестируем функцию

links = getPageLinks(1)
links[:4]

['https://exler.ru/films/nastoyashij-detektiv-2.htm',
 'https://exler.ru/films/nastoyashij-detektiv-1.htm',
 'https://exler.ru/films/horas-i-pit.htm',
 'https://exler.ru/films/hod-korolevy.htm']

Функция работает. Теперь мы теоритически можем достать ммлыки на все фильмы. Теперь посмотрим как достать информацию о конкретном фильме

# Вытаскиваем все данные и показатели на странице

In [37]:
film_page = 'https://exler.ru/films/nastoyashij-detektiv-1.htm'

response = requests.get(film_page, headers = {'User-Agent': UserAgent().safari})

html = response.content
soup = BeautifulSoup(html, 'html.parser')

Попробуем вытащить текстовое описание, кол-во просмотров, кол-во комментариев и дату публикации

In [49]:
# Вытягиваем кусок с датой, кол-вом комментариев и просмотров

data = soup.find(attrs={'class': 'blog-item-date'})
data

<div class="blog-item-date">
                10.11.2014<i class="icon-eye" style="margin-left: 10px"></i> 80142  <a href="#comments"><i class="icon-comments"></i> Комментарии
                    (99)</a>
</div>

In [50]:
# Вытягиваем дату

date = data.get_text(strip = True).split()[0][:10]
date

'10.11.2014'

In [57]:
# Вытягиеваем кол-во просмотров

n_views = data.get_text(strip = True).split('\n')[0].replace('Комментарии', '')[10:]
n_views

'80142'

In [51]:
# Вытягиваем кол-во комменатриев

n_comments = data.get_text(strip = True).split('\n')[1].strip()
n_comments = n_comments.replace('(', '').replace(')', '') # Убираем скобки у комментариев

n_comments

'99'

Полученные данные пока находятся в формате string. Переведём их в нужные форматы после сбора всего датасета. Запишем эти манипуляции в небольшую функцию

In [58]:
def getStats(soup, stats):
    """
        Возвращает дату, число просмотров и кол-во комментариев
        
        soup: объект bs4.BeautifulSoup
            суп текущей страницы

        stats: string
            views/comments/date
    """
    
    try:
        obj = soup.find(attrs={'class': 'blog-item-date'})
        
        if stats == 'date':
            data = obj.get_text(strip = True).split()[0][:10]
            
        elif stats == 'views':
            data = obj.get_text(strip = True).split('\n')[0].replace('Комментарии', '')[10:]
            
        elif stats == 'comments':
            data = obj.get_text(strip = True).split('\n')[1].strip()
            data = data.replace('(', '').replace(')', '')
            
        else:
            print('Вы запросили неверный формат данных')
            
    except:
        obj = None
        
    return data

In [60]:
date = getStats(soup, stats='date')
views = getStats(soup, stats = 'views')
comments = getStats(soup, stats = 'comments')

print(f'Просмотры: {views}\nКомментарии: {comments}\nДата: {date}')

Просмотры: 80142
Комментарии: 99
Дата: 10.11.2014


Функция работает. Теперь соберём текстовую информацию

In [61]:
# Ищем данные с заголовком

data = soup.find(attrs={'class': 'film-title'})
data

<div class="film-title">
            Настоящий детектив                            <span>/</span>
                True Detective                        <div class="blog-item-date">
                10.11.2014<i class="icon-eye" style="margin-left: 10px"></i> 80142  <a href="#comments"><i class="icon-comments"></i> Комментарии
                    (99)</a>
</div>
</div>

In [68]:
# Вытягиваем заголовок

title = data.get_text(strip = True).split('\n')[0].split('/')[0]
title

'Настоящий детектив'

In [72]:
# Ищем данные с текстовым описанием фильма


data = soup.find(attrs={'class': 'film-item-content'})
data


<div class="film-item-content">
<p><a class="fancybox" href="images/10-11-2014/big/1.jpg" rel="media"><img align="left" src="/films/images/10-11-2014/1.jpg"/></a>Главные герои сериала -
      детектив Раст "Расти" Коул (Мэттью
      МакКонахи) и детектив Мартин "Марти"
      Харт (Вуди Харрельсон). Они
      познакомились в 1995 году в Луизиане, где
      их поставили в паре расследовать
      ритуальное убийство проститутки Доры
      Лэнг. На спине убитой была нарисована какая-то
      символика, на голову надета "корона"
      из оленьих рогов.  </p>
<p> Расти считает, что
      должно быть несколько убийств,
      обставленных подобным образом. Марти с
      ним не соглашается. Далее детективам предстоит
      распутать (или не распутать) это и
      некоторые другие загадочные убийства. </p>
<p>Работать в паре им будет очень
      сложно. Расти в свое время пережил
      трагедию в семье, сидел на наркотиках, работал под
      прикрытием в банде наркоторговцев и в конце концов
   

In [73]:
text = data.get_text(strip = True).replace('\r', '').replace('\n', '')
text

'Главные герои сериала -      детектив Раст "Расти" Коул (Мэттью      МакКонахи) и детектив Мартин "Марти"      Харт (Вуди Харрельсон). Они      познакомились в 1995 году в Луизиане, где      их поставили в паре расследовать      ритуальное убийство проститутки Доры      Лэнг. На спине убитой была нарисована какая-то      символика, на голову надета "корона"      из оленьих рогов.Расти считает, что      должно быть несколько убийств,      обставленных подобным образом. Марти с      ним не соглашается. Далее детективам предстоит      распутать (или не распутать) это и      некоторые другие загадочные убийства.Работать в паре им будет очень      сложно. Расти в свое время пережил      трагедию в семье, сидел на наркотиках, работал под      прикрытием в банде наркоторговцев и в конце концов      попал в психиатрическую лечебницу.      После того как он оттуда вышел - парень      выглядит слегка не от мира сего. Он или      постоянно молчит, думая о чем-то своем, а      если Марти просит Р

In [90]:
def getText(soup):
    """
        Возвращает текстовое описание фильма
    """
    
    # название фильма
    obj_1 = soup.find(attrs={'class': 'film-title'})
    title = obj_1.get_text(strip = True).split('\n')[0].split('/')[0].replace('\r', '')
    
    # текстовое описание
    obj_2 = soup.find(attrs={'class': 'film-item-content'})
    text = obj_2.get_text(strip = True).replace('\r', '').replace('\n', '')
    
    return {'Title': title, 'Text': text}
    

Создадим общую функцию возвращающую информацию по текущему фильму

In [91]:
def getFilmData(film_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными

        meme_page: string
            ссылка на страницу с фильмом

    """
    
    # Запрашиваем данные по ссылке
    response = requests.get(film_page, headers = {'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # Если сервер отказал, вернём статус ошибки
        return response.status_code
    
    # Получаеам содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    
    # Текстовые поля
    text_fields = getText(soup = soup)
    
    
    # Используя ранее написанные функции парсим информацию
    date = getStats(soup, stats='date')
    views = getStats(soup, stats = 'views')
    comments = getStats(soup, stats = 'comments')
    
    # Составляем словарь где будут храниться все полученные данные
    data_row = {'date_added': date, 'views': views,
                'comments': comments, 'Title': text_fields['Title'],
                'Text': text_fields['Text']}
    

    
    return data_row

In [92]:
# Тестируем нашу функцию

data_row = getFilmData('https://exler.ru/films/nastoyashij-detektiv-1.htm')

data_row

{'date_added': '10.11.2014',
 'views': '80157',
 'comments': '99',
 'Title': 'Настоящий детектив',
 'Text': 'Главные герои сериала -      детектив Раст "Расти" Коул (Мэттью      МакКонахи) и детектив Мартин "Марти"      Харт (Вуди Харрельсон). Они      познакомились в 1995 году в Луизиане, где      их поставили в паре расследовать      ритуальное убийство проститутки Доры      Лэнг. На спине убитой была нарисована какая-то      символика, на голову надета "корона"      из оленьих рогов.Расти считает, что      должно быть несколько убийств,      обставленных подобным образом. Марти с      ним не соглашается. Далее детективам предстоит      распутать (или не распутать) это и      некоторые другие загадочные убийства.Работать в паре им будет очень      сложно. Расти в свое время пережил      трагедию в семье, сидел на наркотиках, работал под      прикрытием в банде наркоторговцев и в конце концов      попал в психиатрическую лечебницу.      После того как он оттуда вышел - парень      выг

Необходимые данные по странице удачно скачиваются. Теперь попробуем создать свой датасет

# Создание датасета из скачанных данных

In [95]:
import pandas as pd
import numpy as np


film_df = pd.DataFrame(columns = ['date_added', 'views', 'comments', 'Title', 'Text'])

film_df = film_df.append(data_row, ignore_index = True).dropna(axis = 1)

film_df

,date_added,views,comments,Title,Text
0,10.11.2014,80157,99,Настоящий детектив,"Главные герои сериала - детектив Раст ""Ра..."


In [99]:
links

['https://exler.ru/films/nastoyashij-detektiv-2.htm',
 'https://exler.ru/films/nastoyashij-detektiv-1.htm',
 'https://exler.ru/films/horas-i-pit.htm',
 'https://exler.ru/films/hod-korolevy.htm',
 'https://exler.ru/films/roma-1.htm',
 'https://exler.ru/films/bezumnyj-maks-doroga-yarosti.htm',
 'https://exler.ru/films/chernobyl.htm',
 'https://exler.ru/films/delo-richarda-dzhuella.htm',
 'https://exler.ru/films/vo-vse-tyazhkie.htm',
 'https://exler.ru/films/udivitelnaya-missis-mejzel.htm']

In [100]:
# Добавление в датасет данных на странице

import time

for link in links:
    try:
        data_row = getFilmData(link)
        film_df = film_df.append(data_row, ignore_index = True)
        time.sleep(0.3)
    except:
        continue
        

In [101]:
# Просмотр результата

film_df = film_df.drop_duplicates().dropna(axis = 1)

film_df.head()

,date_added,views,comments,Title,Text
0,10.11.2014,80157,99,Настоящий детектив,"Главные герои сериала - детектив Раст ""Ра..."
1,07.12.2015,55459,70,Настоящий детектив 2,"Небольшой промышленный город Винчи, штат Калиф..."
2,10.11.2014,80162,99,Настоящий детектив,"Главные герои сериала - детектив Раст ""Ра..."
3,22.12.2016,40428,46,Хорас и Пит,"""У Хораса и Пита"" - старинный ирландский паб в..."
4,30.11.2020,103916,287,Ход королевы,"Америка, середина пятидесятых. Девятилетняя де..."


Работает. Теперь попробуем спарсить весь сайт

In [107]:
from tqdm import tqdm_notebook

final_df = pd.DataFrame(columns = ['date_added', 'views', 'comments', 'Title', 'Text'])

for page_number in tqdm_notebook(range(1, 196), desc = 'Pages'):
    
    # Собрали ссылки с текущей страницы
    film_links = getPageLinks(page_number)
    for film_links in tqdm_notebook(film_links, desc = 'Films', leave = False):
        # Иногда с первого раза страница не парсится
        for i in range(3):
            try:
                # Собираем данные
                data_row = getFilmData(film_links) 
                # Закидываем данные в таблицу
                final_df = final_df.append(data_row, ignore_index = True)
                # Если всё получилось выходим из внутреннего цикла
                break 
            except:
                # иначе, пробуем ещё несколько раз пока не закончится
                print('Ошибка! Парсим ещё раз: ', film_links)
                continue

Pages:   0%|          | 0/195 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

...

Films:   0%|          | 0/10 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

Films:   0%|          | 0/10 [00:00<?, ?it/s]

TypeError: 'NoneType' object is not subscriptable

In [109]:
final_df.head()

,date_added,views,comments,Title,Text
0,07.12.2015,55462,70,Настоящий детектив 2,"Небольшой промышленный город Винчи, штат Калиф..."
1,10.11.2014,80165,99,Настоящий детектив,"Главные герои сериала - детектив Раст ""Ра..."
2,22.12.2016,40431,46,Хорас и Пит,"""У Хораса и Пита"" - старинный ирландский паб в..."
3,30.11.2020,103921,287,Ход королевы,"Америка, середина пятидесятых. Девятилетняя де..."
4,14.09.2020,74243,29,Рома,"Мексика, город Мехико, 1970 год. Богатый район..."


In [110]:
final_df['Title'].unique()

array(['Настоящий детектив 2', 'Настоящий детектив', 'Хорас и Пит',
       'Ход королевы', 'Рома', 'Безумный Макс: Дорога ярости',
       'Чернобыль', 'Дело Ричарда Джуэлла', 'Во все тяжкие',
       'Удивительная миссис Мейзел', 'Лучше звоните Солу. Шестой сезон',
       'Удивительная миссис Мейзел. 4-й сезон', 'Бумажный дом',
       'Постановка', 'Каратель', '«Питер Пэн» пошел не так',
       'Большая секунда', 'Тед Лассо', 'Наследники',
       'Постановка - 2-й сезон', 'Однажды ночью', 'Двойник', 'Фарго',
       'Босс', 'Баллада Бастера Скраггса', 'Одержимость', 'Банши', '1+1',
       'Домашний арест', 'Мстители', 'Предприятие "Божий дар"', 'Дрянь',
       'Один слуга, два господина', 'Благие знамения', 'Шоу пошло не так',
       'Я жив', 'Дюна', 'Мейр из Исттауна', 'Наследники. Третий сезон',
       'Наследники. Финальный сезон', 'Андор',
       'Отряд самоубийц: Миссия навылет', 'Хэппи', 'Озарк',
       'Что знает Оливия', 'Отец', 'Засланец из космоса', 'Боль и слава',
       'Звук

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1290 entries, 0 to 1289
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date_added  1290 non-null   object
 1   views       1290 non-null   int64 
 2   comments    1290 non-null   int64 
 3   Title       1290 non-null   object
 4   Text        1290 non-null   object
dtypes: int64(2), object(3)
memory usage: 60.5+ KB


In [10]:
# Избавимся от чисел в названии фильма

final_df['Title'] = final_df['Title'].apply(lambda x: re.sub(r'\d', '', x))

final_df['Title'].unique()

array(['Настоящий детектив ', 'Настоящий детектив', 'Хорас и Пит',
       'Ход королевы', 'Рома', 'Безумный Макс: Дорога ярости',
       'Чернобыль', 'Дело Ричарда Джуэлла', 'Во все тяжкие',
       'Удивительная миссис Мейзел', 'Лучше звоните Солу. Шестой сезон',
       'Удивительная миссис Мейзел. -й сезон', 'Бумажный дом',
       'Постановка', 'Каратель', '«Питер Пэн» пошел не так',
       'Большая секунда', 'Тед Лассо', 'Наследники',
       'Постановка - -й сезон', 'Однажды ночью', 'Двойник', 'Фарго',
       'Босс', 'Баллада Бастера Скраггса', 'Одержимость', 'Банши', '+',
       'Домашний арест', 'Мстители', 'Предприятие "Божий дар"', 'Дрянь',
       'Один слуга, два господина', 'Благие знамения', 'Шоу пошло не так',
       'Я жив', 'Дюна', 'Мейр из Исттауна', 'Наследники. Третий сезон',
       'Наследники. Финальный сезон', 'Андор',
       'Отряд самоубийц: Миссия навылет', 'Хэппи', 'Озарк',
       'Что знает Оливия', 'Отец', 'Засланец из космоса', 'Боль и слава',
       'Звук мета

In [11]:
# Избавимся от лишнего слова Комментарии в названии фильмов

final_df['Title'] = final_df['Title'].apply(lambda x: x.replace('Комментарии', ''))
final_df['Title'].unique()

array(['Настоящий детектив ', 'Настоящий детектив', 'Хорас и Пит',
       'Ход королевы', 'Рома', 'Безумный Макс: Дорога ярости',
       'Чернобыль', 'Дело Ричарда Джуэлла', 'Во все тяжкие',
       'Удивительная миссис Мейзел', 'Лучше звоните Солу. Шестой сезон',
       'Удивительная миссис Мейзел. -й сезон', 'Бумажный дом',
       'Постановка', 'Каратель', '«Питер Пэн» пошел не так',
       'Большая секунда', 'Тед Лассо', 'Наследники',
       'Постановка - -й сезон', 'Однажды ночью', 'Двойник', 'Фарго',
       'Босс', 'Баллада Бастера Скраггса', 'Одержимость', 'Банши', '+',
       'Домашний арест', 'Мстители', 'Предприятие "Божий дар"', 'Дрянь',
       'Один слуга, два господина', 'Благие знамения', 'Шоу пошло не так',
       'Я жив', 'Дюна', 'Мейр из Исттауна', 'Наследники. Третий сезон',
       'Наследники. Финальный сезон', 'Андор',
       'Отряд самоубийц: Миссия навылет', 'Хэппи', 'Озарк',
       'Что знает Оливия', 'Отец', 'Засланец из космоса', 'Боль и слава',
       'Звук мета

In [13]:
# Преобразуем колонку date_added в формат date

final_df['date_added'] = pd.to_datetime(final_df['date_added'], format = '%d.%m.%Y')
final_df['date_added'] = final_df['date_added'].dt.strftime('%Y-%m-%d')

In [16]:
final_df['date_added'] = pd.to_datetime(final_df['date_added'], format = '%Y-%m-%d')

In [19]:
# Сохраним датасет

final_df.to_csv('Film_parsing.csv')